In [1]:
# Import all libraries
import os
import numpy as np
import pandas as pd
import matplotlib
import pylab as plt
import random
import h5py


In [2]:
#data dirs
data_dir = './data/'
train_dir_x = os.path.join(data_dir, 'camelyonpatch_level_2_split_train_x.h5')
train_dir_y = os.path.join(data_dir, 'camelyonpatch_level_2_split_train_y.h5')

learned = pd.read_csv(os.path.join(data_dir,'learned.csv'))
learned_index = learned['learned']

not_learned = pd.read_csv(os.path.join(data_dir,'not_learned.csv'))
not_learned_index = not_learned['not_learned']

dirs = [learned_index, not_learned_index]
names = ['learned', 'not_learned']
shapes = [(len(learned_index),96,96,3),(len(not_learned_index),96,96,3)]


In [3]:
def getlengths(dirs):
    lengths = []
    shapes = []
    for d in dirs:
        with h5py.File(d, 'r') as hdf:
            lengths.append(len(hdf['x']))
            shapes.append(hdf['x'].shape)
    return np.array(lengths), shapes
        
    

    
def getDataSample(data_dir, a):
    with h5py.File(data_dir, 'r') as hdf:
        data = []
        for i in a:
            data.append(np.array(list(hdf['x'][i])))
        return np.array(data)
    
def getlbls(data_dir, a):
    with h5py.File(data_dir, 'r') as hdf:
        data = []
        for i in a:
            data.append(np.array(list(hdf['y'][i])))
        return np.array(data)

    
def new_trainin_data_set(dirs, names, shapes):
    #create dir if not exists
    saving_dir = './data/'
    if(not os.path.exists(saving_dir)):
        os.mkdir(saving_dir)
        
    for i in range(len(dirs)):
        #set path
        path = os.path.join(saving_dir, names[i]+"_x.h5")
        path2 = os.path.join(saving_dir, names[i]+"_y.h5")
        
        #create empty dataset
        h5f = h5py.File(path, 'w')
        h5f.create_dataset("x", shapes[i], dtype='uint8')
        h5f.close()
        
        h5f = h5py.File(path2, 'w')
        h5f.create_dataset("y", (len(dirs[i]),1,1,1), dtype='uint8')
        h5f.close()
        
        
        half = int(len(dirs[i])/2)
        
        with h5py.File(path, 'r') as hdf:
            print("path = {}, length = {}, shape = {}".format(path, len(hdf['x']), hdf['x'].shape))
            
        with h5py.File(path2, 'r') as hdf:
            print("path = {}, length = {}, shape = {}".format(path2, len(hdf['y']), hdf['y'].shape))
            
            
        for j in range(2):
            print("j = {}".format(j))
            #get half of the data
            if j == 0:
                data = getDataSample(train_dir_x, dirs[i][:half])
                print('data',data.shape)
                with h5py.File(path, 'a') as hf:
                    hf["x"][:half] = data
                    
                lbls = getlbls(train_dir_y, dirs[i][:half])
                print('lbls',lbls.shape)
                with h5py.File(path2, 'a') as hf:
                    hf["y"][:half] = lbls  
                    
                    
            else: 
                data = getDataSample(train_dir_x, dirs[i][half:])
                print('data',data.shape)
                with h5py.File(path, 'a') as hf:
                    hf["x"][half:] = data
                    
                lbls = getlbls(train_dir_y, dirs[i][half:])
                print('lbls',lbls.shape)
                with h5py.File(path2, 'a') as hf:
                    hf["y"][half:] = lbls

          
            
            
        

In [4]:
new_trainin_data_set(dirs, names, shapes)


path = ./data/learned_x.h5, length = 235211, shape = (235211, 96, 96, 3)
path = ./data/learned_y.h5, length = 235211, shape = (235211, 1, 1, 1)
j = 0
data (117605, 96, 96, 3)
lbls (117605, 1, 1, 1)
j = 1
data (117606, 96, 96, 3)
lbls (117606, 1, 1, 1)
path = ./data/not_learned_x.h5, length = 26933, shape = (26933, 96, 96, 3)
path = ./data/not_learned_y.h5, length = 26933, shape = (26933, 1, 1, 1)
j = 0
data (13466, 96, 96, 3)
lbls (13466, 1, 1, 1)
j = 1
data (13467, 96, 96, 3)
lbls (13467, 1, 1, 1)
